In [3]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline


In [4]:
train = pd.read_csv('data/train.csv')
train.info()
train.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26884 entries, 0 to 26883
Data columns (total 1 columns):
"CUST_COD","LIMIT_BAL","SEX","EDUCATION","MARRIAGE","BIRTH_DATE","PAY_DEC","PAY_NOV","PAY_OCT","PAY_SEP","PAY_AUG","PAY_JUL","BILL_AMT_DEC","BILL_AMT_NOV","BILL_AMT_OCT","BILL_AMT_SEP","BILL_AMT_AUG","BILL_AMT_JUL","PAY_AMT_DEC","PAY_AMT_NOV","PAY_AMT_OCT","PAY_AMT_SEP","PAY_AMT_AUG","PAY_AMT_JUL","DEFAULT PAYMENT JAN"    26884 non-null object
dtypes: object(1)
memory usage: 210.1+ KB


,"""CUST_COD"",""LIMIT_BAL"",""SEX"",""EDUCATION"",""MARRIAGE"",""BIRTH_DATE"",""PAY_DEC"",""PAY_NOV"",""PAY_OCT"",""PAY_SEP"",""PAY_AUG"",""PAY_JUL"",""BILL_AMT_DEC"",""BILL_AMT_NOV"",""BILL_AMT_OCT"",""BILL_AMT_SEP"",""BILL_AMT_AUG"",""BILL_AMT_JUL"",""PAY_AMT_DEC"",""PAY_AMT_NOV"",""PAY_AMT_OCT"",""PAY_AMT_SEP"",""PAY_AMT_AUG"",""PAY_AMT_JUL"",""DEFAULT PAYMENT JAN"""
0,"53613503,1490.68,""M"",""university"",""single"",""#N..."
1,"35524089,5664.584,""M"",""university"",""married"",""..."
2,"58777007,6260.856,""F"",""high school"",""married"",..."
3,"26913418,8944.08,""F"",""graduate school"",""marrie..."
4,"41993633,3577.632,""F"",""graduate school"",""marri..."
5,"46414261,2385.088,""M"",""university"",""single"",""2..."
6,"10438751,596.272,""M"",""university"",""married"",""0..."
7,"19267237,10732.896,""F"",""university"",""single"",""..."
8,"39470148,1490.68,""M"",""university"",""single"",""07..."
9,"24381216,1490.68,""F"",""university"",""married"",""1..."
